In [2]:
import gurobipy as gp
import pandas as pd
from code_map import data_handling, utils, new_meters, final_markets
import numpy as np
import calendar 
from datetime import datetime
import pytz
import openpyxl
import os
import pickle


In [3]:
L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list = data_handling.load_collections("./half_month_collections.pkl")

In [4]:
print(f"Amount of markets : {len(M)}")
print(f"Amount of meters : {len(L)}")
print(f"Amount of meters with direction up or both : {len(L_u)}")
print(f"Amount of meters with direction down or both : {len(L_d)}")
print(f"Amount of hours : {len(H)}")

Amount of markets : 62
Amount of meters : 2189
Amount of meters with direction up or both : 2044
Amount of meters with direction down or both : 2039
Amount of hours : 408


In [ ]:
# Create a new model
half_month_model = gp.Model("AssetToMarket")

# Create decision variables
x = {}
d = {}
y = {}
w = {}
for h in range(len(H)):
    for l in range(len(L)):
        for m in range(len(M)):
            # asset i is connected to market j at hour h
            x[h, l, m] = half_month_model.addVar(lb = 0, ub = 1, vtype=gp.GRB.BINARY, name=f"x_{h}_{l}_{m}")

            d[h,l,m] = 1 if m in compatible_list[h][l] else 0 # compatible_list takes care of both the area constraint and the direction constraint
            
            # adding the constraint
            half_month_model.addConstr(x[h,l,m] <= d[h,l,m])
    for m in range(len(M)):
        # market m has a bid at hour h
        y[h, m] = half_month_model.addVar(lb = 0, ub = 1, vtype=gp.GRB.BINARY, name=f"y_{h}_{m}")
        # market m is activated at hour h
        w[h, m] = half_month_model.addVar(lb = 0, ub = 1, vtype=gp.GRB.BINARY , name=f"w_{h}_{m}")
        

# Set objective            

# Set the objective to maximize the total income expression
half_month_model.setObjective(sum(Ir_hlm[h,l,m] * x[h,l,m] + Ia_hlm[h,l,m] * x[h,l,m] * w[h,m] for h in range(len(H)) for l in range(len(L)) for m in range(len(M)))
                        , gp.GRB.MAXIMIZE) # can possibly remove the x on the activation income

# Add constraints
for h in range(len(H)):
    for l in range(len(L)):
        # Each asset can only be connected to one market at a time
        half_month_model.addConstr(sum(x[h, l, m] for m in range(len(M))) <= 1, f"single_market_for_asset_at_hour_{h}_nr.{l}")
        

    for m, market in enumerate(M):
        
        up_val, down_val = F[h,m]
        if up_val + down_val > 0:
            half_month_model.addConstr(w[h,m] <= y[h,m], f"market_{m}_can_not_be_activated_at_hour_{h}_if_it_is_not_active")
        else:
            half_month_model.addConstr(w[h,m] == 0, f"market_{m}_can_not_be_activated_at_hour_{h}_if_it_is_not_active")
        
        
        # Connect the binary variables by using big M
        #test_model.addConstr(sum(x[h, l, m] for l in range(len(L))) <= len(L) * y[h, m], f"asset_connection_for_hour_{h}_market_{m}")
     
        # Max volume constraint
        
        if market.direction == "up":
            # capacity volume constraint
            half_month_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len(L))) <= Vp_h_m[h,m]  * y[h,m], f"max_volume_for_hour_{h}_market_{m}")
            # activation volume constraint
            half_month_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len(L))) * w[h,m] <= Va_hm[h,m], f"max_volume_for_activation_in-_market_{m}_at_hour_{h}")
            # min volume capacity constraint
            half_month_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len(L))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") 

        elif market.direction == "down":
            # max capacity volume constraint
            half_month_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len(L))) <=  Vp_h_m[h,m]  * y[h,m], f"max_volume_for_hour_{h}_market_{m}")
            # max activation volume constraint
            half_month_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len(L))) * w[h,m] <= Va_hm[h,m], f"max_volume_for_activation_in_market_{m}_at_hour_{h}")
            # min volume capacity constraint
            half_month_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len(L))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") 

        else: # market.direction == "both"
            if dominant_directions[h] == "up":
                # max capacity volume constraint
                half_month_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len(L))) <= Vp_h_m[h,m]  * y[h,m], f"max_volume_for_hour_{h}_market_{m}")
                # max activation volume constraint
                half_month_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len(L))) * w[h,m] <= Va_hm[h,m] , f"max_volume_for_activation_in_market_{m}_at_hour_{h}")
                # min capacity volume constraint
                half_month_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len(L))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") 

            else:
                # max capacity volume constraint
                half_month_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len(L))) <= Vp_h_m[h,m] * y[h,m], f"max_volume_for_hour_{h}_market_{m}")
                # max activation volume constraint
                half_month_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len(L))) * w[h,m] <= Va_hm[h,m], f"max_volume_for_activation_in_market_{m}_at_hour_{h}")
                # min capacity volume constraint
                half_month_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len(L))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") 

    
        
        # The response times for loads l connected to market m cannot exceed the max response time for m
        for l in range(len(L)):
            half_month_model.addConstr(x[h,l,m] * R_h_l[h,l] <= R_m[m] * y[h,m], f"response_time_for_hour_{h}_market_{m}")
        
            
            
            
# Enable logging
half_month_model.setParam('LogFile', 'half_month_test_model.log')

# Solve the model
half_month_model.optimize()
    
if half_month_model.status == gp.GRB.Status.INFEASIBLE:
    half_month_model.computeIIS()


In [ ]:
utils.test_solution_validity(x, y, w, Va_hm, L, M, H)

In [ ]:
new_x_values = {(h, l, m): half_month_model.getVarByName(f"x_{h}_{l}_{m}").X for h in range(len(H)) for l in range(len(L)) for m in range(len(M))}
